In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import torch.backends.cudnn as cudnn
import copy
from torch.autograd import Function
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
import os

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 256
image_size = 28*28
alpha = 0.005
DANN_EPOCHES = 50
DANN_TRAINING_BATCH = 40
dann_path = 'dann.pth'

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Split training set for training and validation
train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_set, val_set = random_split(train_set, [train_size, val_size])

# DataLoader for validation set
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, drop_last = True)
train_loader =  DataLoader(train_set, batch_size=256, shuffle=False, drop_last = True)
test_loader =  DataLoader(test_set, batch_size=256, shuffle=False, drop_last = True)

100%|██████████| 170498071/170498071 [00:01<00:00, 85835463.70it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# DANN

In [4]:
class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [5]:
class DANN(nn.Module):
    def __init__(self):
        super(DANN, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 10),
        )

        self.domain_classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 2),
        )

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, 32, 32)
        feature = self.feature(input_data)
        feature = feature.view(-1, 512)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

In [6]:
def train_DANN(train_loader, model, criterion, optimizer, epoches):
    model.train()
    src_domain_label = torch.ones(batch_size).long().to(device)
    tgt_domain_label = torch.zeros(batch_size).long().to(device)

    for e in range(epoches):
        data_target_iter = iter(train_loader)
        correct_source_domain, correct_tgt_domain = 0, 0
        total = 0
        for i in range(DANN_TRAINING_BATCH):
            # Src
            source, source_label = next(data_target_iter)
            total += source.size(0)

            source, source_label = source.to(device), source_label.to(device)

            class_output, domain_output = model(source, alpha)

            loss_s_label = criterion(class_output, source_label)
            loss_s_domain = criterion(domain_output, src_domain_label)

            _, predicted = torch.max(domain_output.data, 1)
            correct_source_domain += predicted.eq(src_domain_label.data).cpu().sum().item()

            # Tgt
            target, target_label  = next(data_target_iter)
            target, target_label = target.to(device), target_label.to(device)

            class_output, domain_output = model(target, alpha)

            loss_t_label = criterion(class_output, target_label)
            loss_t_domain = criterion(domain_output, tgt_domain_label)

            _, predicted = torch.max(domain_output.data, 1)
            correct_tgt_domain += predicted.eq(tgt_domain_label.data).cpu().sum().item()

            loss = loss_s_label + loss_s_domain + loss_t_domain + loss_t_label
            loss.backward()
            optimizer.step()
        if ((e + 1) % 5 == 0):
            print(f"{e}: source correct: {correct_source_domain/total}, \
                        target correct: {correct_tgt_domain/total}")

In [7]:
if not os.path.exists(dann_path):
    criterion = nn.CrossEntropyLoss()
    dann = DANN().to(device)
    optimizer = optim.Adam(dann.parameters(), lr=0.001)

    train_DANN(train_loader, dann, criterion, optimizer, DANN_EPOCHES)
    with torch.no_grad():
        torch.save(dann.state_dict(), dann_path)
else:
    dann = DANN().to(device)
    dann.load_state_dict(torch.load(dann_path))
    print("Loaded model from file.")

4: source correct: 0.94189453125,                         target correct: 0.0453125
9: source correct: 0.4751953125,                         target correct: 0.519921875
14: source correct: 0.33388671875,                         target correct: 0.69306640625
19: source correct: 0.89814453125,                         target correct: 0.104296875
24: source correct: 0.03935546875,                         target correct: 0.9677734375
29: source correct: 0.990625,                         target correct: 0.0091796875
34: source correct: 0.5033203125,                         target correct: 0.4974609375
39: source correct: 0.02919921875,                         target correct: 0.9734375
44: source correct: 0.983203125,                         target correct: 0.016015625
49: source correct: 0.01865234375,                         target correct: 0.98388671875


# Classifier

In [13]:
class Linear(nn.Module):
    def __init__(self):
        super(Linear, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear (512, 512),
            nn.ReLU(inplace=True),
            nn.Linear (512, 100),
            nn.ReLU(inplace=True),
            nn.Linear (100, 10),
        )
    def forward(self, x):
        x = x.view(-1, 512)
        x = self.classifier(x)

        return x

In [14]:
def test(model, testloader):
    correct = 0
    total = 0

    for images, label in testloader:
        images, label = images.to(device), label.to(device)
        feature = dann.feature(images)
        outputs = model(feature)
        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

    return correct / total

In [21]:
def train_Linear(model, epoches, criterion, optimizer):
    best_model_wts = None
    best_loss = float('inf')
    batch_num = 0

    for inputs, labels in train_loader:
        if (best_model_wts):
            model.load_state_dict(best_model_wts)

        inputs, labels = inputs.to(device), labels.to(device)
        for epoch in range(epoches):
            model.train()
            optimizer.zero_grad()
            with torch.no_grad():
                feature = dann.feature(inputs)

            outputs = model(feature)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for val_inputs, val_labels in val_loader:
                    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                    feature = dann.feature(val_inputs)
                    outputs = model(feature)
                    batch_loss = criterion(outputs, val_labels)
                    val_loss += batch_loss.item()

                if val_loss < best_loss:
                    best_loss = val_loss
                    best_model_wts = copy.deepcopy(model.state_dict())

                test_acc = test(model, test_loader)

            print(f"Batch: {batch_num}, epoch: {epoch}, Train Loss: {loss.item()}, Val Loss: {val_loss}, Test Acc: {test_acc}")
        batch_num += 1

    return best_model_wts

In [ ]:
model = Linear().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

best_model_wts = train_Linear(model, 20, criterion, optimizer)

Batch: 0, epoch: 0, Train Loss: 2.3941524028778076, Val Loss: 182.83107328414917, Test Acc: 0.09985977564102565
Batch: 0, epoch: 1, Train Loss: 4.42768669128418, Val Loss: 209.79903411865234, Test Acc: 0.0999599358974359
Batch: 0, epoch: 2, Train Loss: 5.288842678070068, Val Loss: 116.34704685211182, Test Acc: 0.10016025641025642
Batch: 0, epoch: 3, Train Loss: 2.952768325805664, Val Loss: 105.61661720275879, Test Acc: 0.10016025641025642
Batch: 0, epoch: 4, Train Loss: 2.7232742309570312, Val Loss: 97.7428023815155, Test Acc: 0.10016025641025642
Batch: 0, epoch: 5, Train Loss: 2.540764331817627, Val Loss: 94.71563839912415, Test Acc: 0.10016025641025642
Batch: 0, epoch: 6, Train Loss: 2.4667017459869385, Val Loss: 94.2074568271637, Test Acc: 0.10016025641025642
Batch: 0, epoch: 7, Train Loss: 2.443727970123291, Val Loss: 93.06195664405823, Test Acc: 0.09985977564102565
Batch: 0, epoch: 8, Train Loss: 2.401240348815918, Val Loss: 92.48777365684509, Test Acc: 0.09985977564102565
Batch: 